In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.interpolate import CubicSpline
from itertools import product
from sklearn.metrics import r2_score
import csv
from vme import VME
import itertools

In [41]:
# adjusted R-squared
def r2_score_adjusted(y, y_pred, featurecount):
    n = len(y)
    return 1 - (1-r2_score(y, y_pred)) * (n-1)/(n-featurecount-1)

# AIC
def calculate_aic(y, y_pred, k):
    sse = np.sum((y - y_pred) ** 2)
    n = len(y)
    aic = n * np.log(sse / n) + 2 * k
    return aic

# BIC
def calculate_bic(y, y_pred, k):
    sse = np.sum((y - y_pred) ** 2)
    n = len(y)
    bic = n * np.log(sse / n) + k * np.log(n)
    return bic

# импорт временного ряда

In [42]:
# linux path
# path1 = './'
# path2 = '/'

# windows path
path1 = '.\\'
path2 = '\\'

In [43]:
seasonsTS = ("s10", "s10", "s10", "s10", "s10", "s10", "s10")
trendsTS = ("t1", "t2", "t3", "t4", "t5", "t6", "t7")
noisesTS = ("n1", "n1", "n1", "n1", "n1", "n1", "n1")

cases = ("t1_n1_s10", "t2_n1_s10", "t3_n1_s10", "t4_n1_s10", "t5_n1_s10", "t6_n1_s10", "t7_n1_s10")

In [44]:
Trend = list()
for trend_i in trendsTS:
    t = pd.read_csv(f"{path1}Components{path2}{trend_i}.csv", decimal=',')
    trend = pd.DataFrame(t, dtype=float)
    Trend.append(trend['Value'])
Component_of_TS = Trend[::]
Component_name = "Trend"
range_of_imfs = range(5)

Noise = list()
for noise_i in noisesTS:
    n = pd.read_csv(f"{path1}Components{path2}{noise_i}.csv", decimal=',')
    noise = pd.DataFrame(n, dtype=float)
    Noise.append(noise['Value'])
Component_name = "Noise"
range_of_imfs = range(9, 1, -1)
Component_of_TS = Noise[::]

Season = list()
for season_i in seasonsTS:
    s = pd.read_csv(f"{path1}Components{path2}{season_i}.csv", decimal=',')
    season = pd.DataFrame(s, dtype=float)
    Season.append(season['Value'])
Component_of_TS = Season[::]
Component_name = "Season"
range_of_imfs = range(1,8)



In [45]:
index = list()
TS = list()

data = pd.read_csv(f"{path1}Cases{path2}{cases[0]}.csv", decimal=',')
df = pd.DataFrame(data, dtype=float)
index = df.index

for case_i in cases:
    data = pd.read_csv(f"{path1}Cases{path2}{case_i}.csv", decimal=',')
    df = pd.DataFrame(data, dtype=float)
    TS.append(df.values)


# тестовый анализ

# Тренд

In [61]:
# набор параметров
alpha = (0, 500, 1000, 2500, 5000, 7500, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000) # moderate bandwidth constraint
omega_init = (1, 501, 1001, 2501, 5001, 7501, 10001, 15001, 20001, 25001, 30001, 35001, 40001, 45001, 50001)
tau = (0, 1, 2, 3, 4, 5) # noise-tolerance (no strict fidelity enforcement)
fs = 20000
tol = 1e-6

featurecount = 5


In [67]:
best_params = np.zeros((len(TS), 10), dtype=dict)
best_metriks = np.zeros((len(TS), 4), dtype=dict)

f = np.zeros((len(TS), 1500), dtype=list)
k_i = np.zeros(len(TS), dtype=int)


In [68]:
#  R2
MetricName = "R2"
calculate_metric = lambda y, y_pred: r2_score(y, y_pred)

default_value = -np.inf
maxmin = lambda criteria: np.max(criteria)
arg_maxmin = lambda criteria: np.argmax(criteria)
find_new_best_params = lambda max_criteria, best_metriks: max_criteria > best_metriks


# #  R2Adj
# MetricName = "R2Adj"
# calculate_metric = lambda y, y_pred, featurecount: r2_score_adjusted(y, y_pred, featurecount)


#  AIC
# MetricName = "AIC"
# calculate_metric = lambda y, y_pred, k: calculate_aic(y, y_pred, k)

# default_value = np.inf
# maxmin = lambda criteria: np.min(criteria)
# arg_maxmin = lambda criteria: np.argmin(criteria)
# find_new_best_params = lambda max_criteria, best_metriks: max_criteria < best_metriks


# # BIC
# MetricName = "BIC"
# calculate_metric = lambda y, y_pred, k: calculate_bic(y, y_pred, k)

In [69]:
for i in range(len(TS)):
    best_metriks[i][0] = {"Time Series": f"TS{i+1}", "Component": "Trend", f"{MetricName}": default_value}
    best_metriks[i][1] = {"Time Series": f"TS{i+1}", "Component": "Season", f"{MetricName}": default_value}
    best_metriks[i][2] = {"Time Series": f"TS{i+1}", "Component": "Noise", f"{MetricName}": default_value}
    best_metriks[i][3] = {"Time Series": f"TS{i+1}", "Component": "Noise-freg", f"{MetricName}": default_value}
    
    for j in range(10):
        best_params[i][j] = {"Time Series": f"TS{i+1}", "Metric": f"{MetricName}", "Trend/Season/Noise": None, "IMFs" : f"1 - {j+1}",  "alpha": 0, "tau": 0, "W init": 1}

In [70]:
# для всех лчм
for ts_i in range(len(TS)):
    f[ts_i] = TS[ts_i].reshape((1500,))
    k_i[ts_i] = 0

    best_imfs = 0
    sum_imfs = 0
    isbest_params = True

    while( (k_i[ts_i] < 10) and isbest_params ):
        isbest_params = False

        # для всех значений alpha
        criteria = np.zeros(len(alpha), dtype=float)
        j = 0          # j - индекс значения параметра
        for alpha_i in alpha:
            u, _, _ = VME(f[ts_i], alpha_i, omega_int=1, fs=20000, tau=1, tol=1e-6)

            criteria[j] = calculate_metric(Trend[ts_i], u.T.real + sum_imfs)
            j += 1
        
        max_criteria = maxmin(criteria)
        argmax_criteria = arg_maxmin(criteria)

        if ( find_new_best_params(max_criteria, best_metriks[ts_i][0][f"{MetricName}"]) ):
            best_metriks[ts_i][0][f"{MetricName}"] = max_criteria

            best_params[ts_i][k_i[ts_i]]["IMFs"] = f'1 --- {k_i[ts_i]+1}'
            best_params[ts_i][k_i[ts_i]]["Trend/Season/Noise"] = "Trend"
            best_params[ts_i][k_i[ts_i]]["alpha"] = alpha[argmax_criteria]
            
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], omega_int=1, fs=20000, tau=1, tol=1e-6)
            best_imfs = u.T.real
            isbest_params = True
        

        # для всех значений omega init
        criteria = np.zeros(len(omega_init), dtype=float)
        j = 0
        for omega_init_i in omega_init:
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], omega_init_i, fs=20000, tau=1, tol=1e-6)

            criteria[j] = calculate_metric(Trend[ts_i], u.T.real + sum_imfs)
            j += 1
        
        max_criteria = maxmin(criteria)
        argmax_criteria = arg_maxmin(criteria)

        if ( find_new_best_params(max_criteria, best_metriks[ts_i][0][f"{MetricName}"]) ):
            best_metriks[ts_i][0][f"{MetricName}"] = max_criteria

            best_params[ts_i][k_i[ts_i]]["IMFs"] = f'1 --- {k_i[ts_i]+1}'
            best_params[ts_i][k_i[ts_i]]["Trend/Season/Noise"] = "Trend"
            best_params[ts_i][k_i[ts_i]]["W init"] = omega_init[argmax_criteria]
            
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], best_params[ts_i][k_i[ts_i]]["W init"], fs=20000, tau=1, tol=1e-6)
            best_imfs = u.T.real


        # для всех значений tau
        criteria = np.zeros(len(tau), dtype=float)
        j = 0
        for tau_i in tau:
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], best_params[ts_i][k_i[ts_i]]["W init"], fs=20000, tau=tau_i, tol=1e-6)

            criteria[j] = calculate_metric(Trend[ts_i], u.T.real + sum_imfs)
            j += 1
        
        max_criteria = maxmin(criteria)
        argmax_criteria = arg_maxmin(criteria)

        if ( find_new_best_params(max_criteria, best_metriks[ts_i][0][f"{MetricName}"]) ):
            best_metriks[ts_i][0][f"{MetricName}"] = max_criteria

            best_params[ts_i][k_i[ts_i]]["IMFs"] = f'1 --- {k_i[ts_i]+1}'
            best_params[ts_i][k_i[ts_i]]["Trend/Season/Noise"] = "Trend"
            best_params[ts_i][k_i[ts_i]]["tau"] = tau[argmax_criteria]
            
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], best_params[ts_i][k_i[ts_i]]["W init"], \
                          fs=20000, tau=best_params[ts_i][k_i[ts_i]]["tau"], tol=1e-6)
            
            best_imfs = u.T.real
            isbest_params = True
        


        k_i[ts_i]+=1
        sum_imfs += best_imfs
        if(isbest_params):
            f[ts_i] = f[ts_i] - best_imfs

In [71]:
for ts_i in range(len(TS)):
    best_metriks[ts_i][0][f"{MetricName}"] = format(best_metriks[ts_i][0][f"{MetricName}"], ".5f")

# Сезон

In [72]:
season_k = np.zeros(len(TS), dtype=int)
# для всех лчм
for ts_i in range(len(TS)):
    season_k[ts_i] = k_i[ts_i]
    
    best_imfs = 0
    sum_imfs = 0
    isbest_params = True

    while( (k_i[ts_i] < 10) and isbest_params ):
        isbest_params = False

        # для всех значений alpha
        criteria = np.zeros(len(alpha), dtype=float)
        j = 0          # j - индекс значения параметра
        for alpha_i in alpha:
            u, _, _ = VME(f[ts_i], alpha_i, omega_int=1, fs=20000, tau=1, tol=1e-6)

            criteria[j] = calculate_metric(Season[ts_i], u.T.real + sum_imfs)
            j += 1
        
        max_criteria = maxmin(criteria)
        argmax_criteria = arg_maxmin(criteria)

        if ( find_new_best_params(max_criteria, best_metriks[ts_i][1][f"{MetricName}"]) ):
            best_metriks[ts_i][1][f"{MetricName}"] = max_criteria

            best_params[ts_i][k_i[ts_i]]["IMFs"] = f'{season_k[ts_i]} --- {k_i[ts_i]}'
            best_params[ts_i][k_i[ts_i]]["Trend/Season/Noise"] = "Season"
            best_params[ts_i][k_i[ts_i]]["alpha"] = alpha[argmax_criteria]
            
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], omega_int=1, fs=20000, tau=1, tol=1e-6)
            best_imfs = u.T.real
            isbest_params = True
        

        # для всех значений omega init
        criteria = np.zeros(len(omega_init), dtype=float)
        j = 0
        for omega_init_i in omega_init:
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], omega_init_i, fs=20000, tau=1, tol=1e-6)

            criteria[j] = calculate_metric(Season[ts_i], u.T.real + sum_imfs)
            j += 1
        
        max_criteria = maxmin(criteria)
        argmax_criteria = arg_maxmin(criteria)

        if ( find_new_best_params(max_criteria, best_metriks[ts_i][1][f"{MetricName}"]) ):
            best_metriks[ts_i][1][f"{MetricName}"] = max_criteria

            best_params[ts_i][k_i[ts_i]]["IMFs"] = f'{season_k[ts_i]} --- {k_i[ts_i]}'
            best_params[ts_i][k_i[ts_i]]["Trend/Season/Noise"] = "Season"
            best_params[ts_i][k_i[ts_i]]["W init"] = omega_init[argmax_criteria]
            
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], best_params[ts_i][k_i[ts_i]]["W init"], fs=20000, tau=1, tol=1e-6)
            best_imfs = u.T.real


        # для всех значений tau
        criteria = np.zeros(len(tau), dtype=float)
        j = 0
        for tau_i in tau:
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], best_params[ts_i][k_i[ts_i]]["W init"], fs=20000, tau=tau_i, tol=1e-6)

            criteria[j] = calculate_metric(Season[ts_i], u.T.real + sum_imfs)
            j += 1
        
        max_criteria = maxmin(criteria)
        argmax_criteria = arg_maxmin(criteria)

        if ( find_new_best_params(max_criteria, best_metriks[ts_i][1][f"{MetricName}"]) ):
            best_metriks[ts_i][1][f"{MetricName}"] = max_criteria

            best_params[ts_i][k_i[ts_i]]["IMFs"] = f'{season_k[ts_i]} --- {k_i[ts_i]}'
            best_params[ts_i][k_i[ts_i]]["Trend/Season/Noise"] = "Season"
            best_params[ts_i][k_i[ts_i]]["tau"] = tau[argmax_criteria]
            
            u, _, _ = VME(f[ts_i], best_params[ts_i][k_i[ts_i]]["alpha"], best_params[ts_i][k_i[ts_i]]["W init"], \
                          fs=20000, tau=best_params[ts_i][k_i[ts_i]]["tau"], tol=1e-6)
            
            best_imfs = u.T.real
            isbest_params = True
        


        k_i[ts_i]+=1
        sum_imfs += best_imfs
        if(isbest_params):
            f[ts_i] = f[ts_i] - best_imfs

In [73]:
for ts_i in range(len(TS)):
    best_metriks[ts_i][1][f"{MetricName}"] = format(best_metriks[ts_i][1][f"{MetricName}"], ".5f")

In [74]:
one_dimensional_best_params = list(itertools.chain(*best_params))

best_params_output = [v for v in one_dimensional_best_params if not (v['Trend/Season/Noise'] is None)]
best_params_output

[{'Time Series': 'TS1',
  'Metric': 'R2',
  'Trend/Season/Noise': 'Trend',
  'IMFs': '1 --- 1',
  'alpha': 50000,
  'tau': 0,
  'W init': 15001},
 {'Time Series': 'TS1',
  'Metric': 'R2',
  'Trend/Season/Noise': 'Season',
  'IMFs': '2 --- 2',
  'alpha': 500,
  'tau': 0,
  'W init': 5001},
 {'Time Series': 'TS2',
  'Metric': 'R2',
  'Trend/Season/Noise': 'Trend',
  'IMFs': '1 --- 1',
  'alpha': 50000,
  'tau': 0,
  'W init': 35001},
 {'Time Series': 'TS2',
  'Metric': 'R2',
  'Trend/Season/Noise': 'Season',
  'IMFs': '2 --- 2',
  'alpha': 1000,
  'tau': 0,
  'W init': 1},
 {'Time Series': 'TS3',
  'Metric': 'R2',
  'Trend/Season/Noise': 'Trend',
  'IMFs': '1 --- 1',
  'alpha': 50000,
  'tau': 0,
  'W init': 15001},
 {'Time Series': 'TS3',
  'Metric': 'R2',
  'Trend/Season/Noise': 'Season',
  'IMFs': '2 --- 2',
  'alpha': 500,
  'tau': 0,
  'W init': 5001},
 {'Time Series': 'TS4',
  'Metric': 'R2',
  'Trend/Season/Noise': 'Trend',
  'IMFs': '1 --- 1',
  'alpha': 50000,
  'tau': 0,
  'W i

In [75]:
np.shape(one_dimensional_best_params)

(70,)

In [76]:
np.shape(best_params_output)

(21,)

In [77]:
filename=f"./Output-VME/Parameters-{MetricName}-VME.csv"

fields = best_params[0][0].keys()
fields = list(fields)

with open(filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()  # Write header row

    writer.writerows(best_params_output)  # Write data rows

# Шум

In [78]:
for ts_i in range(len(TS)):
    criteria = calculate_metric(Noise[ts_i], f[ts_i])
    if (find_new_best_params(criteria, best_metriks[ts_i][2][f"{MetricName}"])):
        best_metriks[ts_i][2][f"{MetricName}"] = criteria

# ШУМ freguency

In [79]:
for ts_i in range(len(TS)):
    original_noise = np.abs(np.fft.rfft(Noise[ts_i]))
    imf_freg = np.abs(np.fft.rfft(f[ts_i].astype(float)))

    criteria = calculate_metric(original_noise, imf_freg)
    if (find_new_best_params(criteria, best_metriks[ts_i][3][f"{MetricName}"])):
        best_metriks[ts_i][3][f"{MetricName}"] = criteria

# Вывод

In [80]:
filename=f"./Output-VME/Best_Metriks-{MetricName}-VME.csv"

fields = best_metriks[0][3].keys()
fields = list(fields)

with open(filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()  # Write header row
    for ts_i in range(len(TS)):
        writer.writerows([best_metriks[ts_i][0]])  # Write data rows
        writer.writerows([best_metriks[ts_i][1]])  # Write data rows
        writer.writerows([best_metriks[ts_i][2]])  # Write data rows
        writer.writerows([best_metriks[ts_i][3]])  # Write data rows